- **[準備](#準備)**
- **[実験](#実験)**
    - [パラメータ](#パラメータ)
    - [組み合わせ](#組み合わせ)

# 準備

In [1]:
from chainer.optimizer_hooks import GradientHardClipping as GHC

from deep_learn import *
import Net_box.plane as plane

# 実験

## パラメータ

In [11]:
run = deep_learn()
run.Net = plane.Net

par_arr = [
    [-1., 1.],
    [0., 1.],
    [-1., 0.],
    [-2., 2.],
    [-0.5, 0.5],
    [-0.1, 0.1],
    [-3., 3.],
    [-5., 5.]
]
run.save_dir = "fig/parameter"
i = 1
MSE_test = []
MAE_test = []

run.set_data()

for par in par_arr:
    print("\r%d/%d" % (i, len(par_arr)), end="")
    run.hook_f = GHC(par[0], par[1])
    run.fig_name1 = "MSE of each epoch (parameter is %.1f, %.1f).png" % (par[0], par[1])
    run.fig_name2 = "MAE of each epoch (parameter is %.1f, %.1f).png" % (par[0], par[1])
    run.fit()
    run.plot()
    test_error = run.cal_test_error()
    MSE_test.append(test_error[0])
    MAE_test.append(test_error[1])
    i += 1

8/8

In [12]:
print(par_arr)
print("MSE: ", MSE_test)
print("MAE: ", MAE_test)

[[-1.0, 1.0], [0.0, 1.0], [-1.0, 0.0], [-2.0, 2.0], [-0.5, 0.5], [-0.1, 0.1], [-3.0, 3.0], [-5.0, 5.0]]
MSE:  [variable(138.40294), variable(540.5586), variable(530.9654), variable(158.3173), variable(155.85709), variable(248.41429), variable(152.12968), variable(163.57144)]
MAE:  [variable(7.6048646), variable(19.716103), variable(19.489712), variable(8.066726), variable(8.328745), variable(12.342105), variable(7.896567), variable(8.243191)]


## 組み合わせ

In [14]:
import Net_box.n_layer as n_layer
from chainer.optimizers import AdaBound
import pandas as pd

bs_arr = [100, 150, 200]
net_arr = [
    [2, n_layer.Net2],
    [3, n_layer.Net3],
    [4, n_layer.Net4]
]
lr_arr = [0.01, 0.025, 0.3]
par_arr = [[-i, i] for i in range(1, 6)]

run = deep_learn()
run.optimizer = AdaBound()
i = 1

data = {
    "batch_size": [],
    "num_of_layer": [],
    "lr": [],
    "lower_bound": [],
    "upper_bound": [],
    "MSE_test": [],
    "MAE_test": []
}

run.set_data()

for bs in bs_arr:
    print("\rbs[%d/%d]" % (i, len(bs_arr)), end="")
    run.batch_size = bs
    for net in net_arr:
        print(".", end="")
        run.Net = net[1]
        for lr in lr_arr:
            run.lr = lr
            for par in par_arr:
                run.hook_f = GHC(par[0], par[1])
                run.fit()
                test_error = run.cal_test_error()
                data["batch_size"].append(bs)
                data["num_of_layer"].append(net[0])
                data["lr"].append(lr)
                data["lower_bound"].append(par[0])
                data["upper_bound"].append(par[1])
                data["MSE_test"].append(test_error[0])
                data["MAE_test"].append(test_error[1])
    i += 1

bs[3/3].......

In [15]:
data = pd.DataFrame(data)

data.to_csv("data.csv")